In [30]:
import numpy as np
from numpy.linalg import inv
from numpy import linalg as LA
import matplotlib.pyplot as plt

In [60]:
def createModelMatrix(matrixDim,sparsity):
    #matrixDim=10
    #sparsity = 0.01
    Amat = np.zeros((matrixDim,matrixDim))
    for i in range(0,matrixDim):
        Amat[i,i] = i + 1
    Amat = Amat + sparsity*np.random.randint(20,size=(matrixDim,matrixDim)) 
    print("Amat: \n",Amat)
    u,v=LA.eig(Amat)
    print('Correct roots: ',np.sort(u))
    return Amat

def defineAllVariables(Amat,matrixDim): #defines single vector p,q,eps,H0,V from a given matrix and its dimension
    p=np.zeros((matrixDim,1))
    q=np.zeros((matrixDim,matrixDim-1))
    print(np.shape(p),np.shape(q))
    diagMat=np.diag(Amat)
    print('diagonal of Matrix: ',diagMat)
    
    newDiag=[]
    for i in range(matrixDim):
        for j in range(matrixDim):
            if i==j:
                newDiag.append(Amat[i,j])
                
    print('new diagonal of matrix: ',newDiag)
    
    lowestElements=np.argsort(diagMat)
    print('lowest element index:',lowestElements)
    print('actual lowest element of Hbar:',diagMat[lowestElements[0]])

    p[lowestElements[0],0]=1.0
    print('p',p)
    counter=0
    for i in range(len(lowestElements)):
        if i==lowestElements[0]:
            continue
        else:
            q[lowestElements[i],counter]=1.0
            counter=counter+1


        
        
    H0=np.diag(np.diag(Amat))#np.matmul(np.matmul(projP,Amat),projP)
    V=Amat-H0
    print('V \n',V)
    #print('original Guess:',diagMat[0])
    guessEps=diagMat[lowestElements[0]] +0.01
    eps=guessEps
    print('init Guess:',eps)
    
    return p,q,H0,V,eps

def createT0_projQ(q,qDim,H0Dim,eps,H0,order): #qDim>0
    print('q',q,eps,qDim)
    #eps[0]=eps[0]+0.1
    epsMat=np.zeros((H0Dim,H0Dim))
    denom=np.zeros((H0Dim,H0Dim))
    np.fill_diagonal(epsMat,eps)
    denom=epsMat-H0
    print('eps-H0',denom)
    denom=inv(denom)
    
    sumMatrix=np.zeros((H0Dim,H0Dim))
    for i in range(qDim):
        Q=np.outer(q[:,i],q[:,i])
        sumMatrix=sumMatrix+Q
        
    sumMatrix=denom@sumMatrix
    
#     sumMatrix=np.zeros((H0Dim,H0Dim))
#     for i in range(qDim):
#         Q=np.outer(q[:,i],q[:,i])
#         #print('Q outer:',Q)
#         QH0Q=np.matmul(np.matmul(Q,H0),Q)
#         denom=epsMat-QH0Q

#         for x in range(H0Dim):
#             if (abs(denom[x,x]) < 0.0001):
#                 if (denom[x,x]< 0.00000000000000001):
#                     factor=-1.0
#                     denom[x,x]=factor*0.0001
#                 else:
#                     factor=1.0
#                     denom[x,x]=factor*0.0001
#             denom[x,x]=1.0/denom[x,x]

#         #denom=inv(denom)
    
#             sumMatrix=sumMatrix+np.matmul(np.matmul(Q,denom),Q) #q[:,i],denom),q[:,i].T)
#     print('T0 value is: ',sumMatrix)
    return sumMatrix


def extendPSpace(T0,V,p,order):
    T0V=np.matmul(T0,V)
    tmp=T0V
    
    for i in range(1,order+1):
        print('extending guess for order: ',i)
        tmp=np.matmul(tmp,T0V)
        
    correctionVec=np.matmul(tmp,p)
    print('Correction vector: \n',correctionVec)
    return correctionVec

def computeRHR(fullSpace,H,HDim,order):
    overlap=fullSpace[:,:order+1].T @ fullSpace[:,:order+1]
    print('overlap',overlap)
    s,u=LA.eig(overlap)
    print('little s:',s)
    sDiag=np.zeros((order+1,order+1))#
    np.fill_diagonal(sDiag,s)
    for i in range(order+1):
        sDiag[i,i]=1.0/np.sqrt(sDiag[i,i])
    print('sdiag:',sDiag)
        
    X=u.T @ sDiag
    X=X @ u
    RHR=(fullSpace[:,:order+1].T @ H) @ fullSpace[:,:order+1]
    Hprime=X.T @ RHR
    Hprime=Hprime @ X
    try:
        roots,vecs=LA.eig(Hprime)
        indx=roots.argsort()
        theta=roots[indx]
        vecs=vecs[:,indx] # these are C' ; must convert back to C, ie XC'=C
        transformedVecs=X@vecs # this is the back transform step
        usableRoots=[ elem for elem in theta if elem > 0.15 ]
        usableIndx=[x for x in range(len(theta)) if theta[x] > 0.15]
        print('Usable roots from <R|H|R>: ',usableRoots)
        print('Usable vecs from <R|H|R>: ',transformedVecs[:,usableIndx])
    except np.linalg.LinAlgError as err:
        print('inf or NaN inside the code performing delta^(-1/2)Hdelta^(-1/2) diagonalization!!')
        print("Formal error message: ")
        print(err)
    
    
    ## test orthogonalizing fullspace
    qq,r=LA.qr(fullSpace[:,:order+1])
    RHR=(qq.T @ H) @ qq
    print('RHR: \n',RHR)
    roots,vecs=LA.eig(RHR)
    print('Roots from <R|H|R> via QR:',np.sort(roots))
    indx=roots.argsort()
    theta=roots[indx]
    vecs=vecs[:,indx]
    print('C_i vecs: ',vecs)
    usableRoots=[ elem for elem in theta if elem > 0.15 ]
    usableIndx=[x for x in range(len(theta)) if theta[x] > 0.15]
    print('C_i vecs: ',vecs[:,usableIndx])
    return usableRoots,vecs[:,usableIndx]#usableRoots,transformedVecs[:,usableIndx]   
        
        
def runVariationalAlgo(H,Hdim):
    p,q,H0,V,eps=defineAllVariables(H,Hdim)
    print('H0:',H0)
    fullSpace=np.zeros((Hdim,Hdim))
    print('shape of FS and p',np.shape(fullSpace[:,0]),np.shape(p))
    fullSpace[:,[0]]=p
    order=0
    qDim=Hdim-1
    Hbkup=H
    print('initial guess H0: ',eps)
    finalEps=[]
    for i in range(8):
        print('*********************************')
        print('*********************************')
        print('***** Iteration: ',i)
#         if (order == 1 ): # redefine H0,V from fatPhi of prior iteration
#             projFatPhi=np.outer(fatPhi,fatPhi)
#             print('projFatPhi: ',projFatPhi)
#             H0=(projFatPhi @ H )@projFatPhi
#             V=H-H0
        T0=createT0_projQ(q,qDim,Hdim,eps,H0,order)
        pertVec=extendPSpace(T0,V,p,order) # first order correction
        order=order+1
        fullSpace[:,[order]]=pertVec
        
        ## try orthogonalizing fullSpace
        qq,r=LA.qr(fullSpace[:,:order+1])
        fullSpace[:,:order+1]=qq
        ##
        ritzRoots,ritzVecs=computeRHR(fullSpace,H,Hdim,order)
        print('ritzVecs:',ritzVecs)
        print('ritzRoots:',np.sort(ritzRoots))
        eps=np.sort(ritzRoots)[0]
        finalEps.append(eps)
        fatPhi=fullSpace[:,:order+1]@ritzVecs[:,0]# approx to the lowest energy eigenvector
        
    return finalEps
       



In [61]:
#HeH+
cisHeH=np.loadtxt('cisHeH.txt')
print(cisHeH)
finalCisHeH=np.zeros((11,11)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(11):
    for j in range(i,11):
        print('i,j:',i,j)
        finalCisHeH[i,j]=cisHeH[counter]
        finalCisHeH[j,i]=finalCisHeH[i,j]
        counter=counter+1
        
print('finalCISheH',finalCisHeH)
guessRoots,guessVecs=LA.eig(finalCisHeH)
print('HeH+ CIS roots:',np.sort(guessRoots))

finalEps=runVariationalAlgo(finalCisHeH,11)

[ 8.28692379e-01  5.54853946e-02  9.41294716e-03  0.00000000e+00
  0.00000000e+00  1.00346189e-02  0.00000000e+00  0.00000000e+00
  1.02642480e-02  4.58917500e-03  4.92319800e-02  1.20240609e+00
 -4.05652551e-02  0.00000000e+00  0.00000000e+00 -3.36256815e-02
  0.00000000e+00  0.00000000e+00 -4.23616784e-02 -2.23130733e-02
  9.31659731e-04  1.63686245e+00  0.00000000e+00  0.00000000e+00
 -2.91054360e-03  0.00000000e+00  0.00000000e+00  7.95477687e-03
  1.80429220e-02  2.29463146e-03  1.78808803e+00  0.00000000e+00
  0.00000000e+00 -2.38849123e-03  3.78608309e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.78808803e+00  0.00000000e+00
 -3.78608309e-02 -2.38849123e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  2.12464069e+00  0.00000000e+00  0.00000000e+00
  3.86410672e-02  4.88817502e-02  8.70826803e-03  2.36104099e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  2.36104099e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.01429477e+00 -2.22908

In [20]:
0.81964986

0.81964986

In [62]:
finalEps

[0.8197288932266025,
 0.8196504158764819,
 0.8196501326055231,
 0.8196501312566586,
 0.8196498883488036,
 0.8196498618313413,
 0.8196498618313414,
 0.8196498618313404]

In [63]:
# Water CIS
# Water CIS
water=np.loadtxt('waterCIS.txt')
print(water)
finalwater=np.zeros((100,100)) #dimension of HeH+ CIS matrix; defines i and j index as well

counter=0
for i in range(100):
    for j in range(100):
        #print('i,j:',i,j)
        finalwater[i,j]=water[counter]
        counter=counter+1
        
print('water',finalwater)
guessRoots,guessVecs=LA.eig(finalwater)
print('water CIS roots:',np.sort(guessRoots))
eps=runVariationalAlgo(finalwater,100)
print('Convergence Info: ',eps)

[ 2.03589324e+01  0.00000000e+00  1.26535320e-01 ... -3.97087662e-03
  0.00000000e+00  4.29091660e+01]
water [[ 2.03589324e+01  0.00000000e+00  1.26535320e-01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  2.04157538e+01  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.26535320e-01  0.00000000e+00  2.06988393e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  3.74422163e+00
   0.00000000e+00 -3.97087662e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   3.93693088e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -3.97087662e-03
   0.00000000e+00  4.29091660e+01]]
water CIS roots: [ 0.37098005  0.43294249  0.46643802  0.53947209  0.5917756   0.70385187
  0.86811614  0.91322348  0.95268039  0.95798485  1.04455771  1.10665125
  1.11080485  1.11102898  1.15256459  1.18658206  1.219577    1.23959655
  1.28745578  1.291

In [64]:
print('water CIS info: ',eps)

water CIS info:  [0.37114147143034826, 0.37098262052007414, 0.37098005563641767, 0.370980053759717, 0.3709800537476715, 0.370980053747651, 0.3709800537476514, 0.3709800537476516]
